In [50]:
%load_ext autoreload
%autoreload 2
import os
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from IPython.display import display, HTML
from typing import List
import numpy as np
from copy import deepcopy
import re
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import pandas as pd
import joblib
# from mprompt.config import RESULTS_DIR
import torch.cuda
import json
from sklearn.linear_model import RidgeCV
import pandas as pd
import re
import sklearn.decomposition
import pyLDAvis
import pyLDAvis.lda_model
import numpy as np
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
import adjustText
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import joblib
import os
from os.path import join
from tqdm import tqdm
from mprompt.config import RESULTS_DIR

HUMAN_EVAL_DIR = join(RESULTS_DIR, 'bert', 'human_eval')
# f'b_l{factor_layer}_i{factor_idx}')
fnames = os.listdir(HUMAN_EVAL_DIR)
evals = []
for fname in fnames:
    fdir = os.listdir(join(HUMAN_EVAL_DIR, fname))[0]
    evals.append(joblib.load(join(HUMAN_EVAL_DIR, fname, fdir, 'eval.pkl')))
df = pd.DataFrame([pd.Series(x) for x in evals])
scores_recovered = joblib.load(join(RESULTS_DIR, 'bert', 'wiki_norm_std', 'wiki_our_syn_perc_std_score.pkl'))
df = df[['factor_layer', 'factor_idx', 'explanation_init_strs', 'score_synthetic_std']]
df['score_synthetic_std'] = df['score_synthetic_std'].apply(lambda x: x[0][0]).astype(float)
df['score_recovered'] = df.apply(lambda x: scores_recovered[x['factor_layer']][x['factor_idx']], axis=1)
df = df.rename(
    columns={
        'explanation_init_strs': 'explanation_human',
        'score_synthetic_std': 'score_human',
    }
)
df.to_pickle(join(RESULTS_DIR, 'results_bert_human_eval.pkl'))
df.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,factor_layer,factor_idx,explanation_human,score_human,score_recovered
0,4,13,[Numerical values.],-0.214344,-0.076868
1,10,42,[Something unfortunate happened.],2.427799,1.970631
2,0,30,[left. Adjective or Verb. Mixed senses.],3.681089,5.868305
3,4,47,[plants. Noun. vegetation.],6.263338,5.044046
4,10,152,[In some locations.],-0.410130,0.029060


In [47]:
print(df[['score_human', 'score_recovered']].mean())
# print(df_human[['score_human', 'score_recovered']].sem())

score_human        1.039322
score_recovered    1.603441
dtype: float64


In [49]:
print('win percentage', (df['score_recovered'] > df['score_human']).mean())

win percentage 0.6129032258064516
